# Imports

In [1]:
from __future__ import print_function, division
import scipy.io as sio
import numpy as np

In [2]:
""" Pyomo + Couenne based minimization (global-optimal) """
from pyomo.environ import *
from pyomo.opt import SolverFactory

# Input values

In [3]:
# Set the initial values:
K = 2
M = 3
L = 4
P = 100
sigma2 = 1

In [4]:
p={}
p[0,0] = 33.765777471068674
p[0,1] = 35.5742871072786
p[1,0] = 31.21117311348726
p[1,1] = 29.93557001930011
p[2,0] = 35.02304941544408
p[2,1] = 34.49014287342129

In [5]:
G={}
G[0,0] = 0.7862790964000568
G[0,1] = 0.020164266253024834
G[1,0] = 0.261341066254701
G[1,1] = 0.04353507505992858
G[2,0] = 68.7706562914202
G[2,1] = 0.05991469368813834
G[3,0] = 0.06185885418885737
G[3,1] = 0.5381422638807442
G[4,0] = 0.009355407747506612
G[4,1] = 0.13338736820139793
G[5,0] = 0.045406668801663455
G[5,1] = 0.339859899084291

# Formulate optimization model

In [6]:
# Solve an integer programming problem using Pyomo + COUENNE solver
# Create a concrete model
model = m = ConcreteModel()

## Set parameters and ranges

In [7]:
# Mutable (settable) model parameters:
m.L = Param(initialize=L,within=NonNegativeIntegers)
m.P = Param(initialize=P,within=NonNegativeReals)
m.K = Param(initialize=K,within=NonNegativeReals)
m.M = Param(initialize=M,within=NonNegativeReals)
m.sigma2 = Param(initialize=sigma2,within=NonNegativeReals)

In [8]:
# Model parameter ranges:
m.k = RangeSet(0, m.K-1, within=NonNegativeIntegers)
m.m = RangeSet(0, m.M-1, within=NonNegativeIntegers)
m.km = RangeSet(0, (m.K*m.M)-1, within=NonNegativeIntegers)

In [9]:
# Multi-dimensional model parameters:
m.p = Param(m.m,m.k,initialize=p,within=NonNegativeReals)
m.G = Param(m.km,m.k,initialize=G,within=NonNegativeReals)

## Set the optimization variable

In [10]:
# Set initial values for the multi-dimensional optimization variable 
z={}
z[0,0] = 1
z[0,1] = 0
z[1,0] = 1
z[1,1] = 0
z[2,0] = 1
z[2,1] = 0
z[3,0] = 0
z[3,1] = 1
z[4,0] = 0
z[4,1] = 1
z[5,0] = 0
z[5,1] = 1

In [11]:
# Define and initialize the optimization variable
m.z = Var(m.km,m.k,initialize=z,within=NonNegativeIntegers,bounds=(0,L))

## Formulate the objective function

In [12]:
def objFun(model):
    # Initialize the objective function:
    fun = 0
    # Loop over all cells:
    for k in model.k:
        # Loop over all users:
        for m in model.m:
            interference = 0;
            for l in model.k:
                if l != k:
                    # Inter-cell interference:
                    interference = interference + (1-model.z[model.M*k+m,l]) * model.G[model.M*k+m,l] * model.P
            # Useful signal:
            useful_signal = model.z[model.M*k+m,k] * model.G[model.M*k+m,k] * model.p[m,k]
            
            # Add to the objective function (model.sigma2 is noise):
            fun = fun + log(1+useful_signal/(model.sigma2+interference))/log(2);
    return fun

In [13]:
m.obj = Objective(rule=objFun, sense=maximize)

## Constraints

In [14]:
def ax_constraint_rule(model,i):
    # return the expression for the constraint for i
    return sum(m.z[km,i] for km in m.km) == m.L

In [15]:
m.constr_sum = Constraint(m.k,rule=ax_constraint_rule)

# Execute

## Print model

In [16]:
m.pprint()

3 Set Declarations
    G_index : Dim=0, Dimen=2, Size=12, Domain=None, Ordered=True, Bounds=None
        Virtual
    p_index : Dim=0, Dimen=2, Size=6, Domain=None, Ordered=True, Bounds=None
        Virtual
    z_index : Dim=0, Dimen=2, Size=12, Domain=None, Ordered=True, Bounds=None
        Virtual

3 RangeSet Declarations
    k : Dim=0, Dimen=1, Size=2, Domain=Integers, Ordered=True, Bounds=(0, 1)
        Virtual
    km : Dim=0, Dimen=1, Size=6, Domain=Integers, Ordered=True, Bounds=(0, 5)
        Virtual
    m : Dim=0, Dimen=1, Size=3, Domain=Integers, Ordered=True, Bounds=(0, 2)
        Virtual

7 Param Declarations
    G : Size=12, Index=G_index, Domain=NonNegativeReals, Default=None, Mutable=False
        Key    : Value
        (0, 0) :   0.7862790964000568
        (0, 1) : 0.020164266253024834
        (1, 0) :    0.261341066254701
        (1, 1) :  0.04353507505992858
        (2, 0) :     68.7706562914202
        (2, 1) :  0.05991469368813834
        (3, 0) :  0.06185885418885737

## Initialize the Couenne solver

In [17]:
opt = SolverFactory('couenne')

## Solve the problem

In [18]:
results = opt.solve(m,keepfiles=True,tee=True)

Solver log file: '/tmp/tmpxn_b7eet_couenne.log'
Solver solution file: '/tmp/tmprtur9hww.pyomo.sol'
Solver problem files: ('/tmp/tmprtur9hww.pyomo.nl',)
Couenne 0.5 -- an Open-Source solver for Mixed Integer Nonlinear Optimization
Mailing list: couenne@list.coin-or.org
Instructions: http://www.coin-or.org/Couenne
couenne: 
ANALYSIS TEST: Couenne: new cutoff value -2.1160853273e+01 (0.008417 seconds)
NLP0012I 
              Num      Status      Obj             It       time                 Location
NLP0014I             1      FAILED -54.395858       35 0.007199
NLP0014I             1      FAILED -54.395858       35 0.006882   resolve robustness
NLP3006W Warning : continuing branching, while there are unrecovered failures at nodes
Loaded instance "/tmp/tmprtur9hww.pyomo.nl"
Constraints:            2
Variables:             12 (12 integer)
Auxiliaries:           37 (6 integer)

Coin0506I Presolve 0 (-51) rows, 0 (-49) columns and 0 (-115) elements
Clp0000I Optimal - objective value -6144
Cl

## Save the model .nl file for reference

In [19]:
m.write('mymodel-couenne-integer.nl')

('mymodel-couenne-integer.nl', 140464947593400)

# Results

In [20]:
results

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 0, 'Number of variables': 12, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'couenne\\x3a Optimal', 'Termination condition': 'optimal', 'Id': 3, 'Error rc': 0, 'Time': 0.3355886936187744}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [21]:
m.display()

Model unknown

  Variables:
    z : Size=12, Index=z_index
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        (0, 0) :     0 :   1.0 :     4 : False : False : NonNegativeIntegers
        (0, 1) :     0 :   0.0 :     4 : False : False : NonNegativeIntegers
        (1, 0) :     0 :   0.0 :     4 : False : False : NonNegativeIntegers
        (1, 1) :     0 :   0.0 :     4 : False : False : NonNegativeIntegers
        (2, 0) :     0 :   1.0 :     4 : False : False : NonNegativeIntegers
        (2, 1) :     0 :   1.0 :     4 : False : False : NonNegativeIntegers
        (3, 0) :     0 :   1.0 :     4 : False : False : NonNegativeIntegers
        (3, 1) :     0 :   1.0 :     4 : False : False : NonNegativeIntegers
        (4, 0) :     0 :   0.0 :     4 : False : False : NonNegativeIntegers
        (4, 1) :     0 :   1.0 :     4 : False : False : NonNegativeIntegers
        (5, 0) :     0 :   1.0 :     4 : False : False : NonNegativeIntegers
        (5, 1) :     0 :   1.0

In [22]:
print(value(m.obj.expr))

24.144027344300497
